In [14]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

symbol = 'RELIANCE.NS'

url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol

# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above
page = requests.get(url, headers=headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element
tree.xpath("//h1/text()")

['Reliance Industries Limited (RELIANCE.NS)']

In [15]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

# Ensure that some table rows are found; if none are found, then it's possible
# that Yahoo Finance has changed their page layout, or have detected
# that you're scraping the page.
assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

,0,1,2,3,4
0,Breakdown,3/31/2022,3/31/2021,3/31/2020,3/31/2019
1,Total Assets,"14,996,650,000","13,212,120,000","11,659,150,000","10,024,060,000"
2,Total Liabilities Net Minority Interest,"6,106,810,000","5,217,800,000","7,045,670,000","6,070,140,000"
3,Total Equity Gross Minority Interest,"8,889,840,000","7,994,320,000","4,613,480,000","3,953,920,000"
4,Total Capitalization,"9,671,840,000","8,638,550,000","6,509,630,000","5,946,180,000"
5,Common Stock Equity,"7,794,850,000","7,001,720,000","4,533,320,000","3,871,120,000"
6,Capital Lease Obligations,"156,690,000","83,140,000","86,970,000",NaN
7,Net Tangible Assets,"5,476,870,000","5,551,980,000","3,065,840,000","2,615,680,000"
8,Working Capital,"383,570,000","954,430,000","-1,546,560,000","-865,670,000"
9,Invested Capital,"10,457,900,000","9,519,830,000","7,896,260,000","6,746,170,000"


In [16]:
df = pd.DataFrame(parsed_rows)
df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

# Rename the "Breakdown" column to "Date"
cols = list(df.columns)
cols[0] = 'Date'
df = df.set_axis(cols, axis='columns', inplace=False)

df

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,3/31/2022,"14,996,650,000","6,106,810,000","8,889,840,000","9,671,840,000","7,794,850,000","156,690,000","5,476,870,000","383,570,000","10,457,900,000","5,476,870,000","2,819,740,000","2,301,270,000","6,765,994","6,765,994"
2,3/31/2021,"13,212,120,000","5,217,800,000","7,994,320,000","8,638,550,000","7,001,720,000","83,140,000","5,551,980,000","954,430,000","9,519,830,000","5,551,980,000","2,601,250,000","2,373,050,000","6,762,069","6,762,069"
3,3/31/2020,"11,659,150,000","7,045,670,000","4,613,480,000","6,509,630,000","4,533,320,000","86,970,000","3,065,840,000","-1,546,560,000","7,896,260,000","3,065,840,000","3,449,910,000","3,104,910,000","6,399,357","6,399,357"
4,3/31/2019,"10,024,060,000","6,070,140,000","3,953,920,000","5,946,180,000","3,871,120,000",NaN,"2,615,680,000","-865,670,000","6,746,170,000","2,615,680,000","2,875,050,000","2,836,750,000","6,398,778","6,398,778"


In [17]:
numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

for column_name in numeric_columns:
    df[column_name] = df[column_name].str.replace(',', '') # Remove the thousands separator
    df[column_name] = df[column_name].astype(np.float64) # Convert the column to float64

df.dtypes

Date                                        object
Total Assets                               float64
Total Liabilities Net Minority Interest    float64
Total Equity Gross Minority Interest       float64
Total Capitalization                       float64
Common Stock Equity                        float64
Capital Lease Obligations                  float64
Net Tangible Assets                        float64
Working Capital                            float64
Invested Capital                           float64
Tangible Book Value                        float64
Total Debt                                 float64
Net Debt                                   float64
Share Issued                               float64
Ordinary Shares Number                     float64
dtype: object

In [18]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'DNT': '1', # Do Not Track Request Header 
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

In [19]:
df_income=scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)

In [20]:
df_income

,Date,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,...,Net Interest Income,EBIT,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
1,ttm,6.999620e+09,5.047090e+09,1.952530e+09,1.145900e+09,806630000.0,-145840000.0,841420000.0,162970000.0,607050000.0,...,-145840000.0,987260000.0,5.047090e+09,297970000.0,607050000.0,28360000.0,28360000.0,1.256870e+09,0.0,5492892.0
2,3/31/2022,6.999620e+09,5.047090e+09,1.952530e+09,1.145900e+09,806630000.0,-145840000.0,841420000.0,162970000.0,607050000.0,...,-145840000.0,987260000.0,5.047090e+09,297970000.0,607050000.0,28360000.0,28360000.0,1.256870e+09,0.0,5492892.0
3,3/31/2021,4.669240e+09,3.390610e+09,1.278630e+09,7.369800e+08,541650000.0,-76060000.0,554610000.0,17220000.0,491280000.0,...,-76060000.0,733680000.0,3.390610e+09,265720000.0,491280000.0,106060000.0,106060000.0,8.933400e+08,0.0,3659070.0
4,3/31/2020,5.967430e+09,4.514360e+09,1.453070e+09,7.903600e+08,662710000.0,-96180000.0,536060000.0,137260000.0,393540000.0,...,-96180000.0,734670000.0,4.514360e+09,222030000.0,393540000.0,-26360000.0,-26360000.0,9.830600e+08,0.0,-6234140.0
5,3/31/2019,5.671350e+09,4.436640e+09,1.234710e+09,6.040400e+08,630670000.0,-104980000.0,552270000.0,153900000.0,395880000.0,...,-104980000.0,704740000.0,4.436640e+09,209340000.0,395880000.0,25440000.0,25440000.0,8.886400e+08,0.0,7090128.0


In [37]:
df_income.columns

Index(['Date', 'Total Revenue', 'Cost of Revenue', 'Gross Profit',
       'Operating Expense', 'Operating Income',
       'Net Non Operating Interest Income Expense', 'Pretax Income',
       'Tax Provision', 'Net Income Common Stockholders',
       'Diluted NI Available to Com Stockholders', 'Basic Average Shares',
       'Diluted Average Shares', 'Rent Expense Supplemental', 'Total Expenses',
       'Net Income from Continuing & Discontinued Operation',
       'Normalized Income', 'Interest Income', 'Interest Expense',
       'Net Interest Income', 'EBIT', 'Reconciled Cost of Revenue',
       'Reconciled Depreciation',
       'Net Income from Continuing Operation Net Minority Interest',
       'Total Unusual Items Excluding Goodwill', 'Total Unusual Items',
       'Normalized EBITDA', 'Tax Rate for Calcs',
       'Tax Effect of Unusual Items'],
      dtype='object')

In [28]:
BS_analysis = df# copy columns of dataframe
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales
1,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.301270e+09,6765994.0,6765994.0,8.889840e+09
2,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09
3,3/31/2020,1.165915e+10,7.045670e+09,4.613480e+09,6.509630e+09,4.533320e+09,86970000.0,3.065840e+09,-1.546560e+09,7.896260e+09,3.065840e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613480e+09
4,3/31/2019,1.002406e+10,6.070140e+09,3.953920e+09,5.946180e+09,3.871120e+09,NaN,2.615680e+09,-8.656700e+08,6.746170e+09,2.615680e+09,2.875050e+09,2.836750e+09,6398778.0,6398778.0,3.953920e+09


## Working Capital Per Dollar of Sales = Working Capital ÷ Total Sales
Total Sales is from Income Statement

In [29]:
current_assets = BS_analysis['Total Assets']
current_liabilities = BS_analysis['Total Liabilities Net Minority Interest']
working_capital = current_assets - current_liabilities
working_capital
total_sales = df_income['Total Revenue']
working_capital_per_dollar_of_sales = working_capital / total_sales
BS_analysis['Working Capital per Dollar of Sales'] = working_capital # copy columns of dataframe
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales
1,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.301270e+09,6765994.0,6765994.0,8.889840e+09
2,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09
3,3/31/2020,1.165915e+10,7.045670e+09,4.613480e+09,6.509630e+09,4.533320e+09,86970000.0,3.065840e+09,-1.546560e+09,7.896260e+09,3.065840e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613480e+09
4,3/31/2019,1.002406e+10,6.070140e+09,3.953920e+09,5.946180e+09,3.871120e+09,NaN,2.615680e+09,-8.656700e+08,6.746170e+09,2.615680e+09,2.875050e+09,2.836750e+09,6398778.0,6398778.0,3.953920e+09


## Current Ratio = Current Assets ÷ Current Liabilities

In [30]:
current_ratio = current_assets / current_liabilities
BS_analysis['Current Ratio'] = current_ratio
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales,Current Ratio
1,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.301270e+09,6765994.0,6765994.0,8.889840e+09,2.455726
2,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09,2.532125
3,3/31/2020,1.165915e+10,7.045670e+09,4.613480e+09,6.509630e+09,4.533320e+09,86970000.0,3.065840e+09,-1.546560e+09,7.896260e+09,3.065840e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613480e+09,1.654796
4,3/31/2019,1.002406e+10,6.070140e+09,3.953920e+09,5.946180e+09,3.871120e+09,NaN,2.615680e+09,-8.656700e+08,6.746170e+09,2.615680e+09,2.875050e+09,2.836750e+09,6398778.0,6398778.0,3.953920e+09,1.651372


## debt-to-equity ratio = total liabilities ÷ shareholders' equity

In [32]:
total_liabilities = df['Total Liabilities Net Minority Interest']
shareholders_equity = df['Common Stock Equity']
debt2equity_ratio = total_liabilities / shareholders_equity
BS_analysis['Debt to Equity Ratio'] = debt2equity_ratio
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales,Current Ratio,Debt to Equity Ratio
1,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.301270e+09,6765994.0,6765994.0,8.889840e+09,2.455726,0.783442
2,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09,2.532125,0.745217
3,3/31/2020,1.165915e+10,7.045670e+09,4.613480e+09,6.509630e+09,4.533320e+09,86970000.0,3.065840e+09,-1.546560e+09,7.896260e+09,3.065840e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613480e+09,1.654796,1.554196
4,3/31/2019,1.002406e+10,6.070140e+09,3.953920e+09,5.946180e+09,3.871120e+09,NaN,2.615680e+09,-8.656700e+08,6.746170e+09,2.615680e+09,2.875050e+09,2.836750e+09,6398778.0,6398778.0,3.953920e+09,1.651372,1.568058


## Receivable Turnover = Net Credit Sales ÷ Average Net Receivables for the Period
Net Credit Sales is from Income Statement

In [39]:
net_credit_sales = df_income['Net Income from Continuing Operation Net Minority Interest']
average_net_receivables_for_the_period = df['Tangible Book Value']
receivable_turnover = net_credit_sales / average_net_receivables_for_the_period
BS_analysis['Receivable Turnover'] = receivable_turnover
BS_analysis

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number,Working Capital per Dollar of Sales,Current Ratio,Debt to Equity Ratio,Receivable Turnover
1,3/31/2022,1.499665e+10,6.106810e+09,8.889840e+09,9.671840e+09,7.794850e+09,156690000.0,5.476870e+09,3.835700e+08,1.045790e+10,5.476870e+09,2.819740e+09,2.301270e+09,6765994.0,6765994.0,8.889840e+09,2.455726,0.783442,0.110839
2,3/31/2021,1.321212e+10,5.217800e+09,7.994320e+09,8.638550e+09,7.001720e+09,83140000.0,5.551980e+09,9.544300e+08,9.519830e+09,5.551980e+09,2.601250e+09,2.373050e+09,6762069.0,6762069.0,7.994320e+09,2.532125,0.745217,0.109339
3,3/31/2020,1.165915e+10,7.045670e+09,4.613480e+09,6.509630e+09,4.533320e+09,86970000.0,3.065840e+09,-1.546560e+09,7.896260e+09,3.065840e+09,3.449910e+09,3.104910e+09,6399357.0,6399357.0,4.613480e+09,1.654796,1.554196,0.160243
4,3/31/2019,1.002406e+10,6.070140e+09,3.953920e+09,5.946180e+09,3.871120e+09,NaN,2.615680e+09,-8.656700e+08,6.746170e+09,2.615680e+09,2.875050e+09,2.836750e+09,6398778.0,6398778.0,3.953920e+09,1.651372,1.568058,0.150454


## You can calculate any financial ratios with formulae

Thanks for looking into the Notebook:)
